In [32]:
import json, random, re, collections, itertools, base64, hashlib
from dataclasses import dataclass
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(x))
from pprint import pprint as pp
annot_path = Path("../annotations")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")

## Read mapping

In [2]:
with (data_path/"annotaions.json").open('r') as f:
    annotations = json.load(f)

In [3]:
with (preprocessed_path/"resources.json").open('r') as f:
    resources = json.load(f)
with (preprocessed_path/"ingredients.json").open('r') as f:
    ingredients = json.load(f)
with (preprocessed_path/"labels.json").open('r') as f:
    idx2label = [tuple(t) for t in json.load(f)]
    label2idx={r:i for i,r in enumerate(idx2label)}
ingredient_dict = {k:v for k,v in ingredients}
resource_dict = {k:v for k,v in resources}

In [4]:
states = np.load(str(preprocessed_path/ "ultimate_state.npz" ))
states.files

['104600',
 '109875',
 '116231',
 '116803',
 '116687',
 '104786',
 '102138',
 '117061',
 '103308',
 '106850']

In [5]:
fname = states.files[0]
state = states[fname]
with (preprocessed_path / f"{fname}.json").open('r') as f:
    meta = json.load(f)
state.shape

(8, 17457)

In [33]:
@dataclass
class Instruction:
    ts: int
    command: str
    ingredient: str
    resource:str

In [ ]:
Instruction = collections.namedtuple("Instruction", ("ts", "command", "ingredient", "resource"))
@dataclass
class Instruction:
    ts: int
    command: str
    ingredient: str
    resource:str

In [49]:
def program(annotation, verboze=False):
    max_ts = max(map(int, annotation.keys()))
    new_state = None
    state = {res: set() for res in resource_dict}
    actions = []
    for ts in range(1+max_ts):
        new_state = {res: set() for res in resource_dict}
        for resource, ing in annotation[str(ts)]: 
            new_state[resource].add(ing)
        for resource in new_state.keys():
            added_ings = new_state[resource]-state[resource]
            removed_ings = state[resource]-new_state[resource]
            for ing in added_ings:
                if ing.startswith("L"):
                    if ing!="LIMMEDIATE":
                        actions.append(Instruction(ts, "chef_check", ing, resource))
                elif ing.startswith("T"):
                    actions.append(Instruction(ts, "use", ing, resource))
                else:
                    actions.append(Instruction(ts, "put", ing, resource))
            for ing in removed_ings:
                if ing.startswith("I"):
                    actions.append(Instruction(ts, "remove", ing, resource))
        state = deepcopy(new_state)
    # timestamps in which we only remove things gets appended to the next time stamp
    remove_only_ts = [ts for ts in range(1+max_ts) if any(a for a in actions if a.ts==ts) and not any (a for a in actions if a.ts==ts and a.command!="remove")]
    print(remove_only_ts)
    while any(remove_only_ts):
        for a in actions:
            if a.ts in remove_only_ts:
                a.ts+=1
        remove_only_ts = [ts for ts in range(1+max_ts) if any(a for a in actions if a.ts==ts) and all(a.command=="remove" for a in actions if a.ts==ts)]
        
    # move contents
    for res in resource_dict:
        for ts in range(1, 1+max_ts):
            before = {a.ingredient for a in actions if a.ts<ts and a.command=="put" and a.resource==res and a.ingredient.startswith("I")}
            after = {a.ingredient for a in actions if a.ts==ts and a.command=="remove" and a.resource==res and a.ingredient.startswith("I")}
            new_res = {a.resource for a in actions if a.ts==ts and a.command=="put" and a.resource!=res and a.ingredient in {b for b in before}}
            if len(new_res)!=1:
                continue
            new_res=list(new_res)[0]
            if before<=after:
                actions.append(Instruction(ts, "move_contents", res, new_res))
                actions = [a for a in actions if not (
                    (a.ts==ts and (a.command,a.resource) in {("remove", res), ("put", new_res)} and a.ingredient in before)
                )]
                    
    action_order=["move_contents", "remove", "use", "put", "chef_check"]
    actions = sorted(actions, key=lambda t: (t.ts,action_order.index(t.command)))
    if verboze:
        actions = [(a.ts,a.command,ingredient_dict.get(a.ingredient, resource_dict.get(a.ingredient, "")),resource_dict[a.resource]) for a in actions]
    return actions
pp (program(annotations["104600"][-1], verboze=True))

[]
[(0, 'put', 'chicken wings', 'Sink/Wash'),
 (1, 'move_contents', 'Sink/Wash', 'Sink/Drain'),
 (2, 'remove', 'chicken wings', 'Sink/Drain'),
 (2, 'use', 'Bowl', 'Workspace/Combine'),
 (2, 'put', 'sesame oil', 'Workspace/Combine'),
 (2, 'put', 'chili garlic sauce', 'Workspace/Combine'),
 (2, 'put', 'rice wine', 'Workspace/Combine'),
 (2, 'put', 'cloves garlic', 'Workspace/Combine'),
 (2, 'put', 'white sugar', 'Workspace/Combine'),
 (3, 'put', 'chicken wings', 'Workspace/Combine'),
 (4, 'remove', 'chili garlic sauce', 'Workspace/Combine'),
 (4, 'remove', 'rice wine', 'Workspace/Combine'),
 (4, 'remove', 'white sugar', 'Workspace/Combine'),
 (4, 'remove', 'sesame oil', 'Workspace/Combine'),
 (4, 'remove', 'chicken wings', 'Workspace/Combine'),
 (4, 'remove', 'cloves garlic', 'Workspace/Combine'),
 (4, 'use', 'Skillet', 'Stove/Medium'),
 (4, 'put', 'Oil', 'Stove/Medium'),
 (5, 'put', 'water', 'Stove/Medium'),
 (5, 'put', 'chili garlic sauce', 'Stove/Medium'),
 (5, 'put', 'rice wine', 'St

In [22]:
[].sort

<function list.sort(*, key=None, reverse=False)>